[Assistants migration guide](https://platform.openai.com/docs/assistants/migration)

# Constants and Libraries

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv

load_dotenv("./../../config/credentials_my.env")

True

# Create a client to connecto Azure OpenAI service and deployment

In [2]:
from openai import AzureOpenAI

# Create the client
client = AzureOpenAI(
    # api_key        = os.getenv("AZURE_OPENAI_API_KEY"),  
    # api_version    = os.getenv("AZURE_OPENAI_API_VERSION"), # at least 2024-02-15-preview
    # azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)
print(f"client.base_url: {client.base_url}")

client.base_url: https://mmoaiswc-01.openai.azure.com/openai/


# Check all files already uploaded

In [3]:
def list_uploaded_files (client:AzureOpenAI, delete:bool=False):
    i = 0
    for file in client.files.list().data:
        i += 1
        if delete:
            print(f"File {i}: {file.filename} (id={file.id}) is being deleted...")
            client.files.delete(file.id) # un-comment this line if you want to delete it
        else:
            print(f"File {i}: {file.filename} (id={file.id})")

list_uploaded_files(client=client, delete=True)

File 1: d0e44824-7bf1-4f7f-9e93-489a5105cfd2 (id=assistant-9vVRKnz3hlhNf05bVqYWDhXI) is being deleted...
File 2: 12e3b52b-c194-4192-b5ac-b8ae585713ec (id=assistant-2N35JHzk1dysz9kI03cN5wgg) is being deleted...


# Upload the Assistant file(s)

In [5]:
from openai.types import FileObject
from pathlib import Path

DATA_FOLDER = "./data/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    print(f"Uploading {filePath}...")
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

print(f"file_ids: {file_ids}")

list_uploaded_files(client=client, delete=False)

Uploading ./data/turbines.csv...
file_ids: ['assistant-QRXM4XVFAuorsa2rebBzA5Ez']
File 1: turbines.csv (id=assistant-QRXM4XVFAuorsa2rebBzA5Ez)


# Create an assistant with `code_interpreter` and `file_ids`

In [6]:
assistant = client.beta.assistants.create(
    name="Smart Assistant",
    description="You are a helpful AI assistant who helps answering questions",
    instructions = """
        You are an assistant that can help manage wind turbine farm.
        The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour.
        Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.
        """,
    model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    tools = [{"type": "code_interpreter"}],
    tool_resources = {
        "code_interpreter": {"file_ids": file_ids}
    }
)

assistant

Assistant(id='asst_tNwcnurRacWVUrDPZ7D6oD82', created_at=1738146056, description='You are a helpful AI assistant who helps answering questions', instructions="\n        You are an assistant that can help manage wind turbine farm.\n        The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour.\n        Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.\n        ", metadata={}, model='gpt-4o', name='Smart Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['assistant-QRXM4XVFAuorsa2rebBzA5Ez']), file_search=None), top_p=1.0)

# Create a conversation

## First, create a thread...
Note that `code_interpreter` and `file_search` are empty, because they are associated with the `assistant`, not to the `thread`

In [7]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_5U5eqclB0s5ODf5SS0Srzw63', created_at=1738146105, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## ...then, add a message to the thread

In [8]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the status of turbine 1001 and 1003"
)

thread_messages = client.beta.threads.messages.list(thread.id)

# print(thread_messages.model_dump_json(indent=2))

# Create a Run and check its status

In [11]:
import time, json

run              = client.beta.threads.runs.create(
    thread_id    = thread.id,
    assistant_id = assistant.id,
)

while client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status in ["queued", "in_progress"]:
    print(f"Run status: {run.status}")
    time.sleep(5)

print(f"Final run status: {client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status}\n\n")

# extract the result from the run, if successfull
for m in enumerate(reversed(client.beta.threads.messages.list(thread_id=thread.id).data),1):
    print(f">>> MESSAGE {m[0]}:\n{m[1].content[0].text.value}\n\n")

Run status: queued
Run status: queued
Final run status: completed


>>> MESSAGE 1:
What is the status of turbine 1001 and 1003


>>> MESSAGE 2:
Welcome Turbine Management Assistant.

Let's start by examining the uploaded file to understand its contents and extract the status information for turbines 1001 and 1003.


>>> MESSAGE 3:
The dataset contains information about each turbine, including:

- `Turbine_ID`: The ID of the turbine.
- `Wind_Speed`: The speed of the wind in miles per hour.
- `RPM`: The rotations per minute of the turbine.
- `Voltage`: The output voltage of the turbine in kilovolts.
- `Maintenance_Date`: The last maintenance date in YYYY-MM-DD format.

Now, let's check the status of turbines 1001 and 1003.


>>> MESSAGE 4:
Here is the status of the specified turbines:

- **Turbine 1001**:
  - Wind Speed: 32 mph
  - RPM: 18
  - Voltage: 33 kV
  - Last Maintenance Date: 2023-10-12

- **Turbine 1003**:
  - Wind Speed: 30 mph
  - RPM: 16
  - Voltage: 32 kV
  - Last Maintenan

# Folow-up question below, then run the cell above to calculate and show it!

In [10]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to provide the previous answer"
)

# START teardown

In [12]:
# delete files

files_list = client.files.list().data
print(f"There are {len(files_list)} files to delete")

i = 0
for file in files_list:
    i += 1
    print(f"File {i}/{len(files_list)}: {file.filename} (id={file.id}) is being deleted...")
    client.files.delete(file.id) # un-comment this line if you want to delete it

There are 1 files to delete
File 1/1: turbines.csv (id=assistant-QRXM4XVFAuorsa2rebBzA5Ez) is being deleted...


In [ ]:
print(f"Deleting thread {thread.id}...")
client.beta.threads.delete(thread.id)

In [ ]:
# delete assistants

assistants_list = client.beta.assistants.list().data
print(f"There are {len(assistants_list)} assistants to delete")

i = 0
for assistant in assistants_list:
    i += 1
    print(f"Assistant {i}/{len(assistants_list)}: Assistant {assistant.id}) is being deleted...")
    client.beta.assistants.delete(assistant_id=assistant.id) # un-comment this line if you want to delete it